## Readme Code Examples
These are the code examples from the Readme document.

In [14]:
from pydantic import BaseModel

class Normal(BaseModel):
    boolean: bool
    float: float
    integer: int
    
obj = Normal(
    boolean='yes',
    float='2',
    integer='3',
)
assert obj.boolean is True
assert obj.float == 2.0
assert obj.integer == 3

In [15]:
from pydantic import BaseModel
from fuzztypes import (
    ASCII,
    Datetime,
    Email,
    Fuzzmoji,
    InMemory,
    Integer,
    Person,
    Regex,
    ZipCode,
    flags,
)

# define a source, see EntitySource for using TSV, CSV, JSONL
inventors = ["Ada Lovelace", "Alan Turing", "Claude Shannon"]

# define a named entity type in memory. use OnDisk for larger data sets.
Inventor = InMemory(inventors, search_flag=flags.FuzzSearch)

# custom Regex type for finding twitter handles.
Handle =  Regex(r'@\w{1,15}', examples=["@genomoncology"])

# define a Pydantic class with 9 fuzzy type attriubutes
class Fuzzy(BaseModel):
    ascii: ASCII
    email: Email
    emoji: Fuzzmoji
    handle: Handle
    integer: Integer
    inventor: Inventor
    person: Person
    time: Datetime
    zipcode: ZipCode

# create an instance of class Fuzzy
obj = Fuzzy(
    ascii="άνθρωπος",
    email="John Doe <jdoe@example.com>",
    emoji='thought bubble',
    handle='Ian Maurer (@imaurer)',
    integer='fifty-five',
    inventor='ada luvlace',
    person='mr. arthur herbert fonzarelli (fonzie)',
    time='5am on Jan 1, 2025',
    zipcode="(Zipcode: 12345-6789)",
)

# test the autocorrecting performed

# greek for man: https://en.wiktionary.org/wiki/άνθρωπος
assert obj.ascii == "anthropos"

# extract email via regular expression
assert obj.email == "jdoe@example.com"

# fuzzy match "thought bubble" to "thought balloon" emoji
assert obj.emoji == "💭"

# simple, inline regex example (see above Handle type)
assert obj.handle == "@imaurer"

# convert integer word phrase to integer value
assert obj.integer == 55

# case-insensitive fuzzy match on lowercase, misspelled name
assert obj.inventor == "Ada Lovelace"

# human name parser (title, first, middle, last, suffix, nickname)
assert str(obj.person) == 'Mr. Arthur Herbert Fonzarelli (fonzie)'
assert obj.person.short_name == "Arthur Fonzarelli"
assert obj.person.nickname == "fonzie"
assert obj.person.last == "Fonzarelli"

# convert time phrase to datetime object
assert obj.time.isoformat() == "2025-01-01T05:00:00"

# extract zip5 or zip9 formats using regular expressions
assert obj.zipcode == "12345-6789"

# print JSON on success
print(obj.model_dump_json(indent=4))

{
    "ascii": "anthropos",
    "email": "jdoe@example.com",
    "emoji": "💭",
    "handle": "@imaurer",
    "integer": 55,
    "inventor": "Ada Lovelace",
    "person": {
        "name_format": "{title} {first} {middle} {last} {suffix} ({nickname})",
        "init_format": "{first} {middle} {last}",
        "title": "Mr.",
        "first": "Arthur",
        "middle": "Herbert",
        "last": "Fonzarelli",
        "suffix": "",
        "nickname": "fonzie"
    },
    "time": "2025-01-01T05:00:00",
    "zipcode": "12345-6789"
}
